In [286]:
DATA_LINK = "https://docs.google.com/spreadsheets/d/1LYYyjZtW7cLXMN5mTh3-K1pBRn6ncA_T"
APPEND_TO_LINK = "/gviz/tq?tqx=out:html&tq&gid=1"
TABLE_LINK = DATA_LINK + APPEND_TO_LINK

In [287]:
import pandas as pd

In [288]:
df = pd.read_html(TABLE_LINK, skiprows=2, header=0)[0]

In [289]:
#display(df.head())

In [290]:
def leave_only_number(text):
    return text.split("_")[1]

import re
def parse_links(text):
    links = re.findall(r'(https?://\S+)', str(text).rstrip("."))
    return links

def link_to_badge(links):
    if links:
        links[0] = f"[![Code](https://img.shields.io/badge/Code-159957.svg)]({links[0]})"
        try: 
            links[1] = f"[![Data](https://img.shields.io/badge/Data-20BEFF.svg)]({links[1]})"
        except IndexError:
            pass
        return '<br />'.join(links)

def generate_section_list(f, groups):
    f.writelines('\n## Papers\n\n')
    f.writelines('<details open>\n')
    f.write('<summary>List of sections<a id="sections"></a></summary>\n\n')
    [f.write(title_to_link(name)) for (name, _) in groups]
    f.write('</details>\n\n')

def title_to_link(title):
    return f"- [{title}](#{title.lower().replace(' ', '-')})\n\n"

def generate_sections(f, groups):
    for (name, g_df) in groups:
        f.write(f"## {name}\n\n")
        f.write(generate_section_badges(g_df))
        f.write(g_df[["ID","Title","Links"]].fillna('').to_markdown(colalign = ('center','left','center'), index=False))
        f.write("\n\n")
        f.write(back_to_sections())

import numpy as np
def generate_section_badges(df):
    return f"![Section Papers](https://img.shields.io/badge/Section%20Papers-{df['ID'].count()}-42BA16) ![Papers with Open Code](https://img.shields.io/badge/Papers%20with%20Open%20Code-{df['Links'].replace('', np.nan).count()}-1D7FBF)\n"
    

def back_to_sections():
    return f"[Back to top](#papers)\n\n"

In [291]:
# Generate List of Sections

df = df.rename(columns={"Ms ID #": "ID", "Manuscript Title":"Title", "No. of supplementary files":"Links"})
df["ID"] = df["ID"].apply(lambda cell: leave_only_number(cell))
df["Links"] = df["Links"].apply(lambda cell: link_to_badge(parse_links(cell)))

groups = df.groupby("Section")

with open('generated.md', 'w') as f:
    generate_section_list(f,groups)
    generate_sections(f,groups)

In [292]:
# Merge base.md and generated.md into a single readme.md file

filenames = ["base.md", "generated.md"]
with open('../README.md', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)